In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

d:\Anaconda3\envs\MLOntology\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#global variabls

directory_path = "D:/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)


prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [5]:
print(len(conceptLabelDict))

9955


In [6]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t", 2)
            
            line = splitted[2]
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                tagID = get_trailing_number(splitted[0])
                conceptMappingDict[i]= int(tagID)
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [tagID])

In [7]:
conceptMappingDict={}

train_file = data_path + "ontClassTopology_owl_ncit.txt"

train_corpus = list(read_corpus(train_file))

In [10]:
print(len(train_corpus))
train_corpus[1296:1299]

9955


[TaggedDocument(words=['neoplasm', 'by', 'special', 'category', 'neoplasm', 'by', 'obsolete', 'classification', 'hematopoietic', 'and', 'lymphoid', 'neoplasm', 'antiquated', 'lymphoma', 'by', 'working', 'formulation', 'antiquated', 'intermediate', 'grade', 'lymphoma', 'adult', 'non', 'hodgkin', 'lymphoma', 'grade', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma'], tags=['8131']),
 TaggedDocument(words=['neoplasm', 'by', 'morphology', 'neoplasm', 'by', 'site', 'epithelial', 'neoplasm', 'head', 'and', 'neck', 'neoplasm', 'carcinoma', 'neck', 'neoplasm', 'malignant', 'head', 'and', 'neck', 'neoplasm', 'mucin', 'producing', 'carcinoma', 'laryngeal', 'neoplasm', 'malignant', 'neck', 'neoplasm', 'head', 'and', 'neck', 'carcinoma', 'mucoepide

In [11]:
cores = multiprocessing.cpu_count()

print(cores)
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=1, epochs=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=1, epochs =10, workers=cores),
]


4


In [12]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t4)
Doc2Vec(dm/m,d200,n5,w8,s0.001,t4)


In [13]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 17.6 s
Wall time: 6.44 s


In [14]:
for i, model in enumerate(models):
    temp_path = vector_model_path + "model" + str(i)  # creates a temp file
    model.save(temp_path)

In [ ]:
model = Doc2Vec.load("model1") 

pprint(model.docvecs.most_similar([inferred_vector], topn=20))

In [15]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    pprint(model.docvecs.most_similar([inferred_vector], topn=20))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t4)
[('142823', 0.7881754636764526),
 ('3497', 0.7405018210411072),
 ('4566', 0.7280634045600891),
 ('5671', 0.7232069373130798),
 ('27710', 0.716212809085846),
 ('5662', 0.7042271494865417),
 ('4888', 0.7034896612167358),
 ('4453', 0.7034471035003662),
 ('7527', 0.7026300430297852),
 ('8531', 0.6945658922195435),
 ('5663', 0.6913337111473083),
 ('35875', 0.6851045489311218),
 ('7779', 0.6831587553024292),
 ('3602', 0.6782907247543335),
 ('45612', 0.6752529144287109),
 ('4565', 0.674105167388916),
 ('7454', 0.6707520484924316),
 ('142828', 0.6687768697738647),
 ('4571', 0.6676112413406372),
 ('142829', 0.654915452003479)]
Doc2Vec(dm/m,d200,n5,w8,s0.001,t4)
[('3725', 0.6372205018997192),
 ('45514', 0.6368991136550903),
 ('139545', 0.5913213491439819),
 ('5665', 0.5877687335014343),
 ('45652', 0.5866506099700928),
 ('5658', 0.5679271221160889),
 ('3995', 0.5677281618118286),
 ('90513', 0.5654343366622925),
 ('45631', 0.5635276436805725),
 ('36305', 0.5595

In [ ]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    sims= model.docvecs.most_similar([inferred_vector], topn=20)
    for sim in sims:
        index = str(conceptMappingDict[sim[0]])
        print(conceptLabelDict[index], "score: ", sim[1])
    

In [16]:
model.docvecs['133521']

array([-6.30423054e-03,  2.26436779e-02, -2.37022508e-02,  2.08223797e-02,
        7.71466643e-02, -2.17750873e-02, -1.57992672e-02, -1.73736028e-02,
        2.11519413e-02, -1.98172368e-02, -1.82828847e-02, -2.78533455e-02,
        2.14598104e-02, -2.88935844e-02, -5.85346371e-02,  2.33500190e-02,
        5.70625626e-02, -1.31705217e-02,  2.60410402e-02, -5.64714894e-03,
       -2.05279905e-02, -5.64146824e-02, -9.32227354e-04, -2.97767129e-02,
        7.56343966e-03, -3.85058448e-02, -3.32194790e-02, -7.20774680e-02,
        3.88865396e-02,  1.94361359e-02,  8.27738345e-02,  3.78705794e-03,
        1.08767077e-02,  3.32356058e-03, -6.42525703e-02, -5.77526458e-04,
        7.24956393e-02, -2.97766505e-03,  1.76604334e-02,  3.85918953e-02,
        6.97201176e-06,  5.49658798e-02,  1.97130116e-03, -5.02299704e-03,
        3.27985510e-02,  2.27714051e-02,  3.02614104e-02,  3.52760665e-02,
        3.59978899e-02, -3.52143571e-02,  5.48494644e-02, -3.04640569e-02,
        9.12699325e-04, -

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, iter=5, alpha=0.025, min_alpha=0.025, sample = 1e-3)

gensim.models.doc2vec.Doc2Vec??

model.build_vocab(train_corpus)

len(model.wv.vocab)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)
pprint(sims)

for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

new_model = gensim.models.Doc2Vec.load(temp_path)

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

In [ ]:
print("Total: ",len(train_corpus))


ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    print("Processing: ", doc_id)
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
import random, string

def randomStr(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))


In [ ]:
collections.Counter(ranks) 